# Data Cleaning


Let's import Data and start cleaning Step by Step..

In [1]:
import pandas as pd
df = pd.read_csv(r"C:\Users\giris\OneDrive\Documents\datasets for practice\Reviews.csv")
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


**Checking Null Values**


In [2]:
df.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

So, we have ~560,000 rows, removing 53 missing rows has zero statistical or modeling impact. It’s cleaner and avoids unnecessary text placeholders. So we gonn drop them.


In [3]:
df = df.dropna(subset=['ProfileName', 'Summary'])

In [4]:
print("Shape after dropping nulls:", df.shape)
print("Missing values after drop:\n", df.isna().sum())

Shape after dropping nulls: (568401, 10)
Missing values after drop:
 Id                        0
ProductId                 0
UserId                    0
ProfileName               0
HelpfulnessNumerator      0
HelpfulnessDenominator    0
Score                     0
Time                      0
Summary                   0
Text                      0
dtype: int64


**Feature Selection**


In [5]:
print(df.columns.tolist())

['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text']


We removed unnecessary columns to keep only the ones directly useful for building a recommendation system.This ensures the dataset stays lightweight while retaining all key information about user ratings and review content.


In [7]:
# Keep only the most important columns
df = df[['UserId', 'ProductId', 'Score', 'Time', 'Summary', 'Text']]

print("Shape after column selection:", df.shape)
print(df.head(3))


Shape after column selection: (568401, 6)
           UserId   ProductId  Score        Time                Summary  \
0  A3SGXH7AUHU8GW  B001E4KFG0      5  1303862400  Good Quality Dog Food   
1  A1D87F6ZCVE5NK  B00813GRG4      1  1346976000      Not as Advertised   
2   ABXLMWJIXXAIN  B000LQOCH0      4  1219017600  "Delight" says it all   

                                                Text  
0  I have bought several of the Vitality canned d...  
1  Product arrived labeled as Jumbo Salted Peanut...  
2  This is a confection that has been around a fe...  


**Checking Duplicates**


In [8]:
df.duplicated().sum()

np.int64(831)

We removed 831 duplicate rows to ensure each user–product interaction is unique and the data remains clean.


In [9]:
df = df.drop_duplicates()

**Feature Engineering and Cleaning Data in Text and Summary Column**


We filtered the dataset to keep only valid ratings between 1 and 5, ensuring consistency with Amazon’s rating scale.


In [10]:
df = df[df['Score'].between(1, 5)]

We converted the Time column from UNIX timestamp to readable datetime format for easier temporal analysis.


In [11]:
df['Time'] = pd.to_datetime(df['Time'], unit='s')

we cleaned the Summary and Text columns by removing extra spaces and converting all text to lowercase for consistency.
This standardization helps improve text processing and model accuracy.


In [12]:
df['Summary'] = df['Summary'].str.strip().str.lower()
df['Text'] = df['Text'].str.strip().str.lower()
df.head()


,UserId,ProductId,Score,Time,Summary,Text
0,A3SGXH7AUHU8GW,B001E4KFG0,5,2011-04-27,good quality dog food,i have bought several of the vitality canned d...
1,A1D87F6ZCVE5NK,B00813GRG4,1,2012-09-07,not as advertised,product arrived labeled as jumbo salted peanut...
2,ABXLMWJIXXAIN,B000LQOCH0,4,2008-08-18,"""delight"" says it all",this is a confection that has been around a fe...
3,A395BORC6FGVXV,B000UA0QIQ,2,2011-06-13,cough medicine,if you are looking for the secret ingredient i...
4,A1UQRSCLF8GW1T,B006K2ZZ7K,5,2012-10-21,great taffy,great taffy at a great price. there was a wid...


We have to cleane the Summary and Text columns by removing unwanted characters such as quotes (") and HTML tags (<br />).This ensures the text is neat, uniform, and free from unnecessary formatting, making it easier to read and analyze.

In [13]:
df['Summary'] = df['Summary'].str.replace('"', '', regex=False)
df['Summary'] = df['Summary'].str.replace('<br />', '', regex=False)
df['Summary'] = df['Summary'].str.strip().str.lower()


In [14]:
df['Text'] = df['Text'].str.replace('"', '', regex=False)
df['Text'] = df['Text'].str.replace('<br />', '', regex=False)
df['Text'] = df['Text'].str.strip().str.lower()

We removed reviews exceeding 2000 characters to eliminate excessively long and potentially noisy text entries.
This helped retain concise, meaningful reviews and made the dataset more balanced and easier to analyze.


In [15]:
# Check basic text length statistics
df['text_length'] = df['Text'].str.len()
df['text_length'].describe()

df = df[df['text_length'] <= 2000]

df['text_length'] = df['Text'].str.len()
df['text_length'].describe()
df.head()
df.drop(columns=['text_length'], inplace=True)
df.head()


,UserId,ProductId,Score,Time,Summary,Text
0,A3SGXH7AUHU8GW,B001E4KFG0,5,2011-04-27,good quality dog food,i have bought several of the vitality canned d...
1,A1D87F6ZCVE5NK,B00813GRG4,1,2012-09-07,not as advertised,product arrived labeled as jumbo salted peanut...
2,ABXLMWJIXXAIN,B000LQOCH0,4,2008-08-18,delight says it all,this is a confection that has been around a fe...
3,A395BORC6FGVXV,B000UA0QIQ,2,2011-06-13,cough medicine,if you are looking for the secret ingredient i...
4,A1UQRSCLF8GW1T,B006K2ZZ7K,5,2012-10-21,great taffy,great taffy at a great price. there was a wid...


In [17]:
import re

def clean_text(text):
    text = re.sub(r'<.*?>', ' ', text)            # remove HTML tags
    text = re.sub(r'http\S+', ' ', text)          # remove URLs
    text = re.sub(r'[^a-zA-Z.,!?\'\s]', ' ', text) # keep letters and basic punctuation
    text = re.sub(r'\s+', ' ', text)              # normalize multiple spaces
    return text.strip().lower()


# Apply to both columns
df['Text'] = df['Text'].apply(clean_text)
df['Summary'] = df['Summary'].apply(clean_text)

df['Text'] = df['Text'].str.replace(r'[^\w\s]', '', regex=True)



Let's save the Dataset now.......

In [18]:
cleaned_path = r"C:\Users\giris\OneDrive\Documents\datasets for practice\cleaned_amazon_reviews.csv"
df.to_csv(cleaned_path, index=False)

print(f"Cleaned dataset saved successfully at: {cleaned_path}")


Cleaned dataset saved successfully at: C:\Users\giris\OneDrive\Documents\datasets for practice\cleaned_amazon_reviews.csv


We started by loading the Amazon reviews dataset and removed unwanted columns like Id and ProfileName to keep only the important ones. Then, we dropped missing and duplicate values to make the data clean. We also made sure the ratings were between 1 and 5 and converted the time column into a readable date format.

Next, we cleaned the Summary and Text columns by removing symbols, extra spaces, and HTML tags, and converted everything to lowercase for consistency. Finally, we saved the cleaned dataset, which is now ready for further analysis.